In [18]:
# Kaggle runs 0.21, which has a bug
#!pip install scikit-learn==0.20.3

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn as skl
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, Lasso, LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

import lightgbm as lgb

from xgboost import XGBClassifier, XGBRegressor

from scipy import stats
from scipy.special import boxcox1p

%matplotlib inline

In [21]:
trainCsvFilepath = r'../input/forest-cover-type-prediction/train.csv'
train = pd.read_csv(trainCsvFilepath)

testCsvFilepath = r'../input/forest-cover-type-prediction/test.csv'
test = pd.read_csv(testCsvFilepath)
testIds = test.Id.copy()

In [22]:
merged = pd.concat([train, test], axis=0, sort=False)

# NULL Values

# Feature Engineer

# Transform data

# Encoding

# Drop

# Train and test

train = merged.iloc[:train.shape[0], :]
train;

test = merged.iloc[train.shape[0]:, :]
test;

### Removing outliers recomended by author

In [23]:
xTrain = train.drop(columns=['Cover_Type'])
yTrain = train.loc[:, 'Cover_Type']

In [35]:
xTest = test

In [24]:
subTrain = train.sample(frac=0.1, random_state=42)
print(subTrain.shape[0])
print(subTrain.Cover_Type.value_counts())

1512
3    237
1    227
7    224
2    210
6    207
4    204
5    203
Name: Cover_Type, dtype: int64


In [25]:
subX, subY = train.drop(columns=['Cover_Type']), train.loc[:, 'Cover_Type']

# Scale

scaleFeats = True
if scaleFeats:
    scaler = StandardScaler()
    xTrain = scaler.fit_transform(xTrain)
    xTest = scaler.transform(xTest)

# Dimensionality reduction

performPCA = False

if performPCA:
    pca = PCA(n_components=10, random_state=42)
    pca.fit(xTrain)
    
    print(pca.explained_variance_ratio_)
    
    xTrain = pca.transform(xTrain)
    xTest = pca.transform(xTest)

# Model

In [26]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])

## XGB

xgb = XGBRegressor(random_state=42, n_jobs=-1)

fullPipeline = Pipeline(steps=[
    #('featureEngPipeline', featureEngPipeline),
    #('transformPipeline', transformPipeline),
    ('model', xgb)
])

one_to_left = stats.beta(10, 1)  
from_zero_positive = stats.expon(0, 50)
param_dis = {  
    "model__n_estimators": stats.randint(40, 200),
    "model__max_depth": stats.randint(10, 50),
    "model__learning_rate": stats.uniform(0.05, 0.4),
    #"colsample_bytree": one_to_left,
    #"model__subsample": one_to_left,
    #"model__gamma": stats.uniform(0, 10),
    #'model__reg_alpha': from_zero_positive,
    "model__min_child_weight": stats.randint(1, 7),
}

RSXGB = RandomizedSearchCV(fullPipeline, param_dis, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, cv=5, iid=False)
scores = cross_val_score(RSXGB, xTrain, yTrain, scoring='neg_mean_squared_error', cv=5);

scores.mean()

## LGBM

In [27]:
lg = lgb.LGBMClassifier(random_state=42, n_jobs=-1, objective='multiclass')

fullPipeline = Pipeline(steps=[
    #('featureEngPipeline', featureEngPipeline),
    #('transformPipeline', transformPipeline),
    ('scaler', StandardScaler()),
    ('model', lg)
])

param_dis = {
    'model__max_depth': stats.randint(25, 76),
    'model__learning_rate': stats.uniform(0.01, 0.1),
    'model__num_leaves': stats.randint(300, 1201),
    'model__n_estimator': [200]
}

RSLGB = RandomizedSearchCV(fullPipeline, param_dis, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, cv=5, iid=False)
RSLGB_scores = cross_val_score(RSLGB, xTrain, yTrain, scoring='neg_mean_squared_error', cv=5);
RSLGB_scores.mean()

param_grid = {
    'model__max_depth': [25,50, 75],
    'model__learning_rate': [0.01,0.05,0.1],
    'model__num_leaves': [300,900,1200],
    'model__n_estimators': [200]
}

GSLGB = GridSearchCV(fullPipeline, param_grid, scoring='accuracy', n_jobs=-1, cv=5, iid=False)
GSLGB_scores = cross_val_score(GSLGB, xTrain, yTrain, scoring='accuracy', cv=5);
GSLGB_scores.mean()

In [28]:
lg = lgb.LGBMClassifier(random_state=42, n_jobs=-1, objective='multiclass', 
                        num_leaves=70, max_depth=7, learning_rate=0.1, n_estimators=100)
#lg.fit(xTrain, yTrain)
cross_val_score(lg, subX, subY, cv=5, n_jobs=-1, scoring='accuracy').mean()

0.5400132275132276

In [29]:
param_space =  {'model__max_depth': Categorical([5,25,50]),
    'model__learning_rate': [0.01,0.1],
    'model__num_leaves': [300,1200],
    'model__n_estimators': [50, 200]
}

BSLGB = BayesSearchCV(fullPipeline, param_space, random_state=42, scoring='accuracy', cv=5, n_iter=40, verbose=3, n_jobs=-1)
BSLGB.fit(subX.astype(float), subY)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   38.8s remaining:   58.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   41.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.4s remaining:   11.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.8s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.3s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s finished


BayesSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))]),
       fit_params=None, iid=True, n_iter=40, n_jobs=-1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=42,
       refit=True, return_train_score=False, scoring='accuracy',
       search_spaces={'model__max_depth': Categorical(categories=(5, 25, 50), prior=None), 'model__learning_rate': [0.01, 0.1], 'model__num_leaves': [300, 1200], 'model__n_estimators': [50, 200]},
       verbose=3)

In [34]:
print(BSLGB.best_params_)
cv_results_df = pd.DataFrame(BSLGB.cv_results_)
print(cv_results_df.mean_test_score.max())

{'model__learning_rate': 0.04183987669962862, 'model__max_depth': 5, 'model__n_estimators': 50, 'model__num_leaves': 300}
0.5788359788359788


## SVM

svm = SVC(kernel='linear')
cross_val_score(svm, xTrain, yTrain, cv=3, n_jobs=-1, scoring='accuracy').mean()

# Submission

In [31]:
model = BSLGB.best_estimator_

In [ ]:
%timeit model.fit(xTrain.astype(float), yTrain)

In [37]:
pred = model.predict(xTest)

C:\Anaconda3x64\envs\ml\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [38]:
output=pd.DataFrame({'Id':testIds,'Cover_Type':pred})

In [39]:
output.to_csv('submission.csv', index=False)

In [40]:
output.head()

,Id,Cover_Type
0,15121,2
1,15122,2
2,15123,1
3,15124,1
4,15125,1
